# Make correlation templates for SDM

In [1]:
import nibabel as nib
import numpy as np
from nilearn import datasets, image

In [2]:
data = datasets.fetch_oasis_vbm()

/Users/taylor/Documents/tsalo/nilearn/nilearn/datasets/struct.py:774: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


In [3]:
img_4d = image.concat_imgs(data["gray_matter_maps"])

In [4]:
img_4d.shape

(91, 109, 91, 403)

In [5]:
img_4d_data = img_4d.get_fdata()
corr_4d = np.zeros(img_4d.shape[:3] + (3,))
img_4d_data_std = np.std(img_4d_data, axis=3)
xyz = np.vstack(np.where(img_4d_data_std)).T

for dim in range(3):
    for coord in range(xyz.shape[0]):
        x, y, z = xyz[coord, :]
        if dim == 0:
            x_s, y_s, z_s = x + 1, y, z
        elif dim == 1:
            x_s, y_s, z_s = x, y + 1, z
        elif dim == 2:
            x_s, y_s, z_s = x, y, z + 1

        if (x_s >= img_4d_data.shape[0]) or (y_s >= img_4d_data.shape[1]) or (z_s >= img_4d_data.shape[2]):
            corr = 0
        else:
            static_1d = img_4d_data[x, y, z, :]
            shifted_1d = img_4d_data[x_s, y_s, z_s, :]
            corr = np.corrcoef(static_1d, shifted_1d)[0, 1]
        
        corr_4d[x, y, z, dim] = corr

corr_4d[np.isnan(corr_4d)] = 0

/opt/miniconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/miniconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [6]:
corr_img = nib.Nifti1Image(corr_4d, img_4d.affine, img_4d.header)
corr_img.to_filename("gray_matter_correlations.nii.gz")